In [0]:
from core import *
import networkx as nx
import time
import pandas as pd

# Leitura das instâncias 

- Instância toy1 (toy1_instance): grafo valorado com custos positivos e não direcionado. Exemplo de entrada
para ser usada pelos algoritmos de caminho mínimo e árvore geradora mínima.

In [0]:
toy1_instance = {
    "a": {"b": 60, "c": 54, "d": 42},
    "b": {"a": 60, "d": 71, "f": 29},
    "c": {"a": 54, "d": 56, "e": 67},
    "d": {"a": 42, "b": 71, "c": 56, "f": 52, "g": 87, "e": 26},
    "e": {"c": 67, "d": 26, "g": 70, "i": 73},
    "f": {"b": 29, "d": 52, "g": 20, "h": 25},
    "g": {"d": 87, "e": 70, "f": 20, "h": 36, "j": 32, "i": 59},
    "h": {"f": 25, "g": 36, "j": 25},
    "i": {"e": 73, "g": 59, "j": 26},
    "j": {"g": 32, "h": 25, "i": 26},
}

- Instância toy2: grafo de fluxo valorado com custos positivos e direcionado. Exemplo de
entrada para ser usada pelo algoritmo de fluxo (Ford-Fulkerson).

In [0]:
toy2_instance = {
    "1": {"2": 3, "3": 3},
    "2": {"3": 1, "4": 3, "5": 2},
    "3": {"4": 5},
    "4": {"6": 6},
    "5": {"4": 3, "6": 3},
}

Instâncias de benchmarks disponíveis na literatura: exemplos de entrada para serem aplicadas
aos algoritmos de caminho mínimo e árvore geradora mínima.
Os algoritmos devem ser executados com pelo menos 3 diferentes instâncias do site
http://www.dis.uniroma1.it/challenge9/download.shtml. Neste site existe uma tabela com arquivos
de dados referentes a 12 redes de estradas dos USA.

Sugiro escolher as três menores: New York
(NY), San Francisco (BAY) e Colorado (COL). Existem dois tipos de rede para cada cidade citada
acima: uma delas possui informações de distância (coluna Distance graph da tabela) e a outra,
tempo de viagem (coluna Travel Time graph da tabela). Sugiro aplicar suas implementações apenas
nas redes de distâncias. Informações do formato dos arquivos de dados podem ser encontradas em
http://www.dis.uniroma1.it/challenge9/format.shtml.

In [0]:
ny_path = "USA-road-d.NY.gr.gz"
san_path = "USA-road-d.BAY.gr.gz"
col_path = "USA-road-d.COL.gr.gz"

In [0]:
download_data(
    "http://www.diag.uniroma1.it/challenge9/data/USA-road-d/USA-road-d.NY.gr.gz",
    ny_path,
)

download_data(
    "http://www.diag.uniroma1.it/challenge9/data/USA-road-d/USA-road-d.BAY.gr.gz",
    san_path,
)
download_data(
    "http://www.diag.uniroma1.it//challenge9/data/USA-road-d/USA-road-d.COL.gr.gz",
    col_path,
)

## Arquivo de resultados

In [0]:
try:
    results = pd.read_csv("results.csv")
except Exception as e:
    results = pd.DataFrame(
        columns=[
            "instancia",
            "n",
            "m",
            "algoritmo",
            "custo",
            "tempo",
            "resultado",
            "anteriores",
        ]
    )

In [0]:
results

# Plot dos gráficos mais simples

## Toy 1

In [0]:
# Create an empty graph
graph_toy1_instance = nx.Graph()

# Add nodes from adjacency list
graph_toy1_instance.add_nodes_from(toy1_instance.keys())

# Add edges from adjacency list
for node, neighbors in toy1_instance.items():
    for neighbor, weight in neighbors.items():
        graph_toy1_instance.add_edge(node, neighbor, weight=weight)

In [0]:
plot_graph(graph_toy1_instance)

## Toy 2

In [0]:
graph_toy2_instance = nx.DiGraph()

# Add nodes from adjacency list
graph_toy2_instance.add_nodes_from(toy2_instance.keys())

# Add edges from adjacency list
for node, neighbors in toy2_instance.items():
    for neighbor, weight in neighbors.items():
        graph_toy2_instance.add_edge(node, neighbor, weight=weight)


In [0]:
plot_graph(graph_toy2_instance, arrows=True)

# Djikstra

````python

vINI = vértice inicial
d(vINI, vINI) = 0
d(vINI, i) = INFINITO, para todo i, i pertence a V – {vINI}
fechado = vazio
aberto = V
anterior(i) = 0, para todo i, i pertence a V
enquanto(aberto ≠ vazio) {
    k = vértice pertencente a aberto, mais próximo do vértice inicial
    fechado = fechado + k; 
    aberto = aberto – k;
    para cada vizinho i de k que está em aberto faça{
        custo = min{d(vINI, i), d(vINI,k) + c(k,i)}

        se (custo < d(vINI, i)) então
            d(vINI, i) = custo; anterior(i) = k

    }
}

````

## TOY 1

In [0]:
start_node = "a"
start_time = time.time()
distances, previous = dijkstra(toy1_instance, start_node)
end_time = time.time()

In [0]:
print("Time between function executions:", end_time - start_time, "seconds")

In [0]:
results_list = [
    "toy1",
    graph_toy1_instance.number_of_nodes(),
    graph_toy1_instance.number_of_edges(),
    "Dijkstra",
    sum(distances.values()),
    end_time - start_time,
    distances,
    previous
]

results.loc[len(results)] = results_list

## New york

In [0]:
graph = read_gr_graph(ny_path.replace(".gz", ''))

In [0]:
graph_ny = nx.to_dict_of_dicts(graph)

In [0]:
graph_ny = {
    node1: {
        node2: subdict['weight'] for node2, subdict in nested_dict.items()
    } for node1, nested_dict in graph_ny.items()
}

In [0]:
graph_ny

In [0]:
start_node = 1
start_time = time.time()
distances, previous = dijkstra(graph_ny, '1')
end_time = time.time()
print("Time between function executions:", end_time - start_time, "seconds")

In [0]:
distances_df = pd.DataFrame({'Node': list(distances.keys()), 'Distance': list(distances.values())})
distances_df.to_csv('distances_ny.csv', index=False)

previous_df = pd.DataFrame({'Node': list(previous.keys()), 'Previous': list(previous.values())})
previous_df.to_csv('previous_ny.csv', index=False)

In [0]:
distances_df = pd.read_csv('distances_ny.csv')
previous_df = pd.read_csv('previous_ny.csv')

In [0]:
column1 = distances_df['Node'].tolist()
column2 = distances_df['Distance'].tolist()
distances = dict(zip(column1, column2))
total_cost = sum(distances_df['Distance'])

In [0]:
column1 = previous_df['Node'].tolist()
column2 = previous_df['Previous'].tolist()
previous = dict(zip(column1, column2))

In [0]:
results_list = [
    "new_york",
    graph.number_of_nodes(),
    graph.number_of_edges(),
    "Dijkstra",
    total_cost,
    end_time - start_time,
    distances,
    previous
]

results.loc[len(results)] = results_list

In [0]:
results

## San Francisco

In [0]:
graph = read_gr_graph(san_path.replace(".gz", ''))

In [0]:
graph_sa = nx.to_dict_of_dicts(graph)

In [0]:
graph.number_of_edges(), graph.number_of_nodes()

In [0]:
graph_sa = {
    node1: {
        node2: subdict['weight'] for node2, subdict in nested_dict.items()
    } for node1, nested_dict in graph_sa.items()
}

In [0]:
start_node = '1'
start_time = time.time()
distances, previous = dijkstra(graph_sa, start_node)
end_time = time.time()
print("Time between function executions:", end_time - start_time, "seconds")

In [0]:
distances_df = pd.DataFrame({'Node': list(distances.keys()), 'Distance': list(distances.values())})
distances_df.to_csv('distances_san.csv', index=False)

previous_df = pd.DataFrame({'Node': list(previous.keys()), 'Previous': list(previous.values())})
previous_df.to_csv('previous_san.csv', index=False)

In [0]:
distances_df = pd.read_csv('distances_san.csv')
previous_df = pd.read_csv('previous_san.csv')

In [0]:
column1 = distances_df['Node'].tolist()
column2 = distances_df['Distance'].tolist()
distances = dict(zip(column1, column2))
total_cost = sum(distances_df['Distance'])

In [0]:
column1 = previous_df['Node'].tolist()
column2 = previous_df['Previous'].tolist()
previous = dict(zip(column1, column2))

In [0]:
results_list = [
    "san_francisco",
    graph.number_of_nodes(),
    graph.number_of_edges(),
    "Dijkstra",
    total_cost,
    end_time - start_time,
    distances,
    previous
]

results.loc[len(results)] = results_list

In [0]:
results

## Colorado

In [0]:
graph = read_gr_graph(col_path.replace(".gz", ''))

In [0]:
graph_col = nx.to_dict_of_dicts(graph)

In [0]:
graph_col = {
    node1: {
        node2: subdict['weight'] for node2, subdict in nested_dict.items()
    } for node1, nested_dict in graph_col.items()
}

In [0]:
start_node = '1'
start_time = time.time()
distances, previous = dijkstra(graph_col, start_node)
end_time = time.time()
print("Time between function executions:", end_time - start_time, "seconds")

In [0]:
distances_df = pd.DataFrame({'Node': list(distances.keys()), 'Distance': list(distances.values())})
distances_df.to_csv('distances_col.csv', index=False)

previous_df = pd.DataFrame({'Node': list(previous.keys()), 'Previous': list(previous.values())})
previous_df.to_csv('previous_col.csv', index=False)

In [0]:
distances_df = pd.read_csv('distances_col.csv')
previous_df = pd.read_csv('previous_col.csv')

In [0]:
column1 = distances_df['Node'].tolist()
column2 = distances_df['Distance'].tolist()
distances = dict(zip(column1, column2))
total_cost = sum(distances_df['Distance'])

In [0]:
column1 = previous_df['Node'].tolist()
column2 = previous_df['Previous'].tolist()
previous = dict(zip(column1, column2))

In [0]:
results_list = [
    "colorado",
    graph.number_of_nodes(),
    graph.number_of_edges(),
    "Dijkstra",
    total_cost,
    end_time - start_time,
    distances,
    previous
]

results.loc[len(results)] = results_list

In [0]:
results.to_csv("results.csv", index=False)

In [0]:
results

# PRIM

````python
entrada: G = (V,E), Lista de Adjacência de G: A(v), v pertence a V, matriz de pesos

T ← vazio;
V' ← {u};

para-todo v pertencente a V – V' faça
L(v) ← peso ({u,v});
fim-para-todo

enquanto V' diferente de V faça
    ache um vértice w tal que L(w) = min {L(v)| v pertencente a V-V'};
    u = o vértice de V', ligado a w, representando a aresta com o menor custo;
    e = {u,w};
    T ← T U {e};
    V' ← V'U {w};
    para-todo v pertencente a V – V' faça
        se peso({v,w}) < L(v) então
            L(v) ← p({v,w});
        fim-se
    fim-para-todo
fim-enquanto
saída: T

````

## Toy 1

In [0]:
start_time = time.time()
minimum_spanning_tree, total_cost = kruskal(toy1_instance)
end_time = time.time()
print("Time between function executions:", end_time - start_time, "seconds")

In [0]:
minimum_spanning_tree_toy1_instance = nx.Graph()

In [0]:
for u, v, weight in minimum_spanning_tree:
    minimum_spanning_tree_toy1_instance.add_edge(u, v, weight=weight)

In [0]:
plot_graph(minimum_spanning_tree_toy1_instance, figsize=(12, 8))

In [0]:
minimum_spanning_tree_toy1_instance_dict = nx.to_dict_of_dicts(minimum_spanning_tree_toy1_instance)


minimum_spanning_tree_toy1_instance_dict = {
    node1: {
        node2: subdict['weight'] for node2, subdict in nested_dict.items()
    } for node1, nested_dict in minimum_spanning_tree_toy1_instance_dict.items()
}

In [0]:
results_list = [
    "toy_1",
    minimum_spanning_tree_toy1_instance.number_of_nodes(),
    minimum_spanning_tree_toy1_instance.number_of_edges(),
    "Kruskal",
    total_cost,
    end_time - start_time,
    minimum_spanning_tree_toy1_instance_dict,
    ""
]

results.loc[len(results)] = results_list

In [0]:
results

## New york

In [0]:
start_time = time.time()
minimum_spanning_tree, total_cost = kruskal(graph_ny)
end_time = time.time()
print("Time between function executions:", end_time - start_time, "seconds")

In [0]:
total_cost

In [0]:
minimum_spanning_tree_new_york = nx.Graph()

In [0]:
for u, v, weight in minimum_spanning_tree:
    minimum_spanning_tree_new_york.add_edge(u, v, weight=weight)

In [0]:
minimum_spanning_tree_new_york_dict = nx.to_dict_of_dicts(minimum_spanning_tree_new_york)

minimum_spanning_tree_new_york_dict = {
    node1: {
        node2: subdict['weight'] for node2, subdict in nested_dict.items()
    } for node1, nested_dict in minimum_spanning_tree_new_york_dict.items()
}

In [0]:
results_list = [
    "new_york",
    minimum_spanning_tree_new_york.number_of_nodes(),
    minimum_spanning_tree_new_york.number_of_edges(),
    "Kruskal",
    total_cost,
    end_time - start_time,
    minimum_spanning_tree_new_york_dict,
    ""
]

results.loc[len(results)] = results_list


## San Francisco

In [0]:
start_time = time.time()
minimum_spanning_tree, total_cost = kruskal(graph_sa)
end_time = time.time()
print("Time between function executions:", end_time - start_time, "seconds")

In [0]:
minimum_spanning_tree_san = nx.Graph()

In [0]:
for u, v, weight in minimum_spanning_tree:
    minimum_spanning_tree_san.add_edge(u, v, weight=weight)

In [0]:
minimum_spanning_tree_san_dict = nx.to_dict_of_dicts(minimum_spanning_tree_san)


minimum_spanning_tree_san_dict = {
    node1: {node2: subdict["weight"] for node2, subdict in nested_dict.items()}
    for node1, nested_dict in minimum_spanning_tree_san_dict.items()
}

In [0]:
results_list = [
    "san_francisco",
    minimum_spanning_tree_san.number_of_nodes(),
    minimum_spanning_tree_san.number_of_edges(),
    "Kruskal",
    total_cost,
    end_time - start_time,
    minimum_spanning_tree_san_dict,
    ""
]


In [0]:
results.loc[len(results)] = results_list

## Colorado

In [0]:
start_time = time.time()
minimum_spanning_tree, total_cost = kruskal(graph_col)
end_time = time.time()
print("Time between function executions:", end_time - start_time, "seconds")

In [0]:
minimum_spanning_tree_colorado = nx.Graph()

In [0]:
for u, v, weight in minimum_spanning_tree:
    minimum_spanning_tree_colorado.add_edge(u, v, weight=weight)

In [0]:
minimum_spanning_tree_colorado_dict = nx.to_dict_of_dicts(minimum_spanning_tree_colorado)


minimum_spanning_tree_colorado_dict = {
    node1: {
        node2: subdict['weight'] for node2, subdict in nested_dict.items()
    } for node1, nested_dict in minimum_spanning_tree_colorado_dict.items()
}

In [0]:
results_list = [
    "colorado",
    minimum_spanning_tree_colorado.number_of_nodes(),
    minimum_spanning_tree_colorado.number_of_edges(),
    "Kruskal",
    total_cost,
    end_time - start_time,
    minimum_spanning_tree_colorado_dict,
    ""
]


In [0]:
results.loc[len(results)] = results_list

In [0]:
results.to_csv("results.csv")

# Ford-Fulkerson

## Toy 2

##DIMACS

Instância de entrada para o algoritmo de fluxo: um exemplo de entrada a ser aplicado ao
algoritmo de Ford-Fulkerson
Considerar o exemplo do arquivo no formato DIMACS representando o digrafo de 6 vértices e 8
arcos apresentado em https://lpsolve.sourceforge.net/5.5/DIMACS_maxf.htm.